In [1]:
!apt-get update
!apt-get install -y openjdk-8-jdk
!java -version

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,798 kB]
Get:13 https://developer.downloa

In [2]:
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz

!tar -xzf hadoop-3.3.6.tar.gz

--2025-03-15 13:00:59--  https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M   110MB/s    in 8.4s    

2025-03-15 13:01:27 (82.9 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]



In [3]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
os.environ["PATH"] = f"{os.environ['HADOOP_HOME']}/bin:{os.environ['PATH']}"

!echo $JAVA_HOME
!echo $HADOOP_HOME
!hadoop version

/usr/lib/jvm/java-8-openjdk-amd64
/content/hadoop-3.3.6
Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /content/hadoop-3.3.6/share/hadoop/common/hadoop-common-3.3.6.jar


In [6]:
%%bash
cat > $HADOOP_HOME/etc/hadoop/core-site.xml << 'EOL'
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://localhost:9000</value>
    </property>
</configuration>
EOL

cat > $HADOOP_HOME/etc/hadoop/hdfs-site.xml << 'EOL'
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
    <property>
        <name>dfs.namenode.name.dir</name>
        <value>/content/hdfs/namenode

bash: line 22: warning: here-document at line 12 delimited by end-of-file (wanted `EOL')


In [7]:
# Start NameNode
!$HADOOP_HOME/bin/hdfs --daemon start namenode

# Start DataNode
!$HADOOP_HOME/bin/hdfs --daemon start datanode

# Verifikasi daemons berjalan
!jps

4752 Jps


In [8]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
  print(f'File yang terupload: {filename}')
  print(f'Ukuran: {len(uploaded[filename])} bytes')

Saving wordcount.txt to wordcount.txt
File yang terupload: wordcount.txt
Ukuran: 1498 bytes


In [9]:
!cat wordcount.txt

UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA TAHUN 1945
PEMBUKAAN

Bahwa sesungguhnya Kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu, maka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan perikeadilan.


Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat yang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan pintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil dan makmur.


Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan luhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan dengan ini kemerdekaannya.


Kemudian daripada itu untuk membentuk suatu Pemerintah Negara Indonesia yang melindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk memajukan kesejahteraan umum, mencerdaskan kehidupan bangsa, dan ikut melaksanakan ketertiban dunia yang berdasarkan kemerdekaan, perdamaian

In [10]:
%%writefile mapper.py
#!/usr/bin/env python3

import sys
import re

for line in sys.stdin:
    line = line.strip()
    line = line.lower()
    line = re.sub(r'[^a-z0-9\s]', ' ', line)
    words = line.split()

    for word in words:
        print(f'{word}\t1')

Writing mapper.py


In [11]:
%%writefile reducer.py
#!/usr/bin/env python3

import sys

current_word = None
current_count = 0
word = None

for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)

    try:
        count = int(count)
    except ValueError:
        continue

    if current_word == word:
        current_count += count
    else:
        if current_word:
            print(f'{current_word}\t{current_count}')
        current_count = count
        current_word = word

if current_word == word:
    print(f'{current_word}\t{current_count}')

Writing reducer.py


In [12]:
!chmod +x mapper.py
!chmod +x reducer.py

In [13]:
import os
streaming_jar = f"{os.environ['HADOOP_HOME']}/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar"

!ls -la {streaming_jar}

-rw-r--r-- 1 1000 1000 141267 Jun 18  2023 /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar


In [14]:
!find $HADOOP_HOME -name "hadoop-streaming*.jar"

/content/hadoop-3.3.6/share/hadoop/tools/sources/hadoop-streaming-3.3.6-test-sources.jar
/content/hadoop-3.3.6/share/hadoop/tools/sources/hadoop-streaming-3.3.6-sources.jar
/content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar


In [24]:
streaming_jar = "/content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar"

!ls -la {streaming_jar}

-rw-r--r-- 1 1000 1000 141267 Jun 18  2023 /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar


In [25]:
!$HADOOP_HOME/bin/hdfs dfs -rm -r -f /user/colab/output_streaming

!$HADOOP_HOME/bin/hadoop jar {streaming_jar} \
  -files mapper.py,reducer.py \
  -mapper "python3 mapper.py" \
  -reducer "python3 reducer.py" \
  -input /user/colab/input/wordcount.txt \
  -output /user/colab/output_streaming

2025-03-15 13:19:51,302 ERROR conf.Configuration: error parsing conf hdfs-site.xml
com.ctc.wstx.exc.WstxEOFException: Unexpected EOF; was expecting a close tag for element <value>
 at [row,col,system-id]: [11,0,"file:/content/hadoop-3.3.6/etc/hadoop/hdfs-site.xml"]
	at com.ctc.wstx.sr.StreamScanner.throwUnexpectedEOF(StreamScanner.java:701)
	at com.ctc.wstx.sr.BasicStreamReader.throwUnexpectedEOF(BasicStreamReader.java:5590)
	at com.ctc.wstx.sr.BasicStreamReader.nextFromTree(BasicStreamReader.java:2791)
	at com.ctc.wstx.sr.BasicStreamReader.next(BasicStreamReader.java:1121)
	at org.apache.hadoop.conf.Configuration$Parser.parseNext(Configuration.java:3405)
	at org.apache.hadoop.conf.Configuration$Parser.parse(Configuration.java:3191)
	at org.apache.hadoop.conf.Configuration.loadResource(Configuration.java:3084)
	at org.apache.hadoop.conf.Configuration.loadResources(Configuration.java:3045)
	at org.apache.hadoop.conf.Configuration.loadProps(Configuration.java:2923)
	at org.apache.hadoop.

ADA ERROR MASALAH DISTREAMING JAR

In [26]:
!cat $HADOOP_HOME/etc/hadoop/hdfs-site.xml

<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
    <property>
        <name>dfs.namenode.name.dir</name>
        <value>/content/hdfs/namenode


In [29]:
import os
hadoop_home = os.environ.get('HADOOP_HOME')
print(f"Nilai HADOOP_HOME: {hadoop_home}")

Nilai HADOOP_HOME: /content/hadoop-3.3.6


In [30]:
%%writefile /content/hadoop-3.3.6/etc/hadoop/hdfs-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
    <property>
        <name>dfs.namenode.name.dir</name>
        <value>/content/hdfs/namenode</value>
    </property>
    <property>
        <name>dfs.datanode.data.dir</name>
        <value>/content/hdfs/datanode</value>
    </property>
</configuration>

Overwriting /content/hadoop-3.3.6/etc/hadoop/hdfs-site.xml


In [31]:
!pkill -f "hdfs"
!/content/hadoop-3.3.6/bin/hdfs --daemon stop datanode
!/content/hadoop-3.3.6/bin/hdfs --daemon stop namenode

import time
time.sleep(5)

!mkdir -p /content/hdfs/namenode
!mkdir -p /content/hdfs/datanode

!/content/hadoop-3.3.6/bin/hdfs namenode -format -force

!/content/hadoop-3.3.6/bin/hdfs --daemon start namenode
!/content/hadoop-3.3.6/bin/hdfs --daemon start datanode

!jps

2025-03-15 13:23:00,542 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = 0c97970be51e/172.28.0.12
STARTUP_MSG:   args = [-format, -force]
STARTUP_MSG:   version = 3.3.6
STARTUP_MSG:   classpath = /content/hadoop-3.3.6/etc/hadoop:/content/hadoop-3.3.6/share/hadoop/common/lib/gson-2.9.0.jar:/content/hadoop-3.3.6/share/hadoop/common/lib/jersey-servlet-1.19.4.jar:/content/hadoop-3.3.6/share/hadoop/common/lib/netty-handler-ssl-ocsp-4.1.89.Final.jar:/content/hadoop-3.3.6/share/hadoop/common/lib/netty-transport-4.1.89.Final.jar:/content/hadoop-3.3.6/share/hadoop/common/lib/netty-transport-native-kqueue-4.1.89.Final-osx-aarch_64.jar:/content/hadoop-3.3.6/share/hadoop/common/lib/jsr311-api-1.1.1.jar:/content/hadoop-3.3.6/share/hadoop/common/lib/netty-codec-4.1.89.Final.jar:/content/hadoop-3.3.6/share/hadoop/common/lib/jettison-1.5.4.jar:/content/hadoop-3.3.6/share/hadoop/common/lib/zookeeper-

In [32]:
!jps

10260 Jps
9785 NameNode
9839 DataNode


In [34]:
streaming_jar = "/content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar"

!/content/hadoop-3.3.6/bin/hdfs dfs -rm -r -f /user/colab/output_streaming

!/content/hadoop-3.3.6/bin/hadoop jar {streaming_jar} \
  -files mapper.py,reducer.py \
  -mapper "python3 mapper.py" \
  -reducer "python3 reducer.py" \
  -input /user/colab/input/wordcount.txt \
  -output /user/colab/output_streaming

2025-03-15 13:24:52,541 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-03-15 13:24:52,815 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-03-15 13:24:52,815 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-03-15 13:24:52,862 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-03-15 13:24:53,569 INFO mapred.FileInputFormat: Total input files to process : 1
2025-03-15 13:24:53,718 INFO mapreduce.JobSubmitter: number of splits:1
2025-03-15 13:24:54,157 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local215390605_0001
2025-03-15 13:24:54,157 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-15 13:24:54,892 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/job_local215390605_0001_620df2d1-3e0e-4f3e-9e5e-d38462ffc694/mapper.py
2025-03-15 13:24:55,003 INFO mapred.LocalDistributedCacheMana

In [45]:
!/content/hadoop-3.3.6/bin/hdfs dfs -ls /user/colab/output_streaming


Found 2 items
-rw-r--r--   1 root supergroup          0 2025-03-15 13:27 /user/colab/output_streaming/_SUCCESS
-rw-r--r--   1 root supergroup       1362 2025-03-15 13:27 /user/colab/output_streaming/part-00000


In [46]:
!/content/hadoop-3.3.6/bin/hdfs dfs -cat /user/colab/output_streaming/part-00000


1945	1
Allah	1
Atas	1
Bahwa	1
DASAR	1
Dan	1
Dasar	1
Esa,	1
INDONESIA	1
Indonesia	9
Indonesia,	2
Indonesia.	1
Keadilan	1
Kebangsaan	1
Kemanusiaan	1
Kemerdekaan	2
Kemudian	1
Kerakyatan	1
Ketuhanan	1
Kuasa	1
Maha	2
NEGARA	1
Negara	4
PEMBUKAAN	1
Pemerintah	1
Permusyawaratan/Perwakilan,	1
Persatuan	1
REPUBLIK	1
Republik	1
TAHUN	1
UNDANG-UNDANG	1
Undang-Undang	1
Yang	2
abadi	1
adil	2
atas	1
bagi	1
bangsa	2
bangsa,	1
bebas,	1
beradab,	1
berbahagia	1
berdasar	1
berdasarkan	1
berdaulat,	1
berkat	1
berkedaulatan	1
berkehidupan	1
bersatu,	1
dalam	3
dan	10
darah	1
daripada	1
dengan	6
depan	1
di	1
didorongkan	1
dihapuskan,	1
dipimpin	1
disusunlah	1
dunia	2
gerbang	1
hak	1
harus	1
hikmat	1
ialah	1
ikut	1
ini	1
itu	3
itu,	1
karena	1
ke	1
keadilan	1
kebangsaan	1
kebijaksanaan	1
kehidupan	1
keinginan	1
kemerdekaan	2
kemerdekaan,	1
kemerdekaannya.	1
kepada	2
kesejahteraan	1
ketertiban	1
luhur,	1
maka	3
makmur.	1
melaksanakan	1
melindungi	1
memajukan	1
membentuk	1
mencerdaskan	1
mengantarkan	1
menyatakan

In [47]:
from google.colab import files
files.download('output_word_count.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
from google.colab import files
files.download('mapper.py')

from google.colab import files
files.download('reducer.py')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>